# ANN-Pytorch-Diabetes Dataset

In [4]:
import numpy as np
import pandas as pd
import torch



In [5]:
data = pd.read_csv('diabetes.csv')

In [6]:
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


## Data Preprocessing

In [9]:
data.shape


(768, 8)

In [16]:
X = data.iloc[:,:-1].values
y_string= list(data.iloc[:,-1])

In [17]:
X

array([[  6. , 148. ,  72. , ...,   0. ,  33.6,  50. ],
       [  1. ,  85. ,  66. , ...,   0. ,  26.6,  31. ],
       [  8. , 183. ,  64. , ...,   0. ,  23.3,  32. ],
       ...,
       [  5. , 121. ,  72. , ..., 112. ,  26.2,  30. ],
       [  1. , 126. ,  60. , ...,   0. ,  30.1,  47. ],
       [  1. ,  93. ,  70. , ...,   0. ,  30.4,  23. ]])

In [22]:
#y_string

In [34]:
# neural network only understand numbers! So convert the string to labels
y= []
for string in y_string:
    if string == 'positive':
        y.append(1)
    else:
        y.append(0)

In [23]:
#y

In [36]:
y = np.array(y, dtype = 'float64')

In [24]:
#y

In [38]:
y.shape

(768,)

## Data Normalization

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [25]:
#X

In [42]:
## convert the arrays to PyTorch tensors


X = torch.tensor(X)
y = torch.tensor(y) 

print(X.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768])


In [43]:
X = torch.tensor(X)
y = torch.tensor(y).unsqueeze(1)   #Adds a dimension to convert this array to 2D

print(X.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


C:\Users\Vikee\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
C:\Users\Vikee\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


## Creating and loading the dataset - Custom dataset

torch.utils.data.Dataset is an abstract class representing a dataset. 


we inherit from this class and build custom dataset


Your custom dataset should inherit Dataset and override the following methods:

 ###              __ len __

 ###             __ getitem __
 
 ###             __ init __ 

In [44]:
from torch.utils.data import Dataset

In [45]:
class Dataset(Dataset):   # inherit the Datset
    def __init__(self,X,y):    # initialization function
        self.X = X    # associate the objects to X and y . i.e the inputs and output
        self.y = y
        
    def __getitem__(self,index):    # Get one item from the dataset
        return self.X[index], self.y[index]
    
    def __len__(self): # length of dataset
        return len(self.X)

In [46]:
# create a object to call the custom dateset class

dataset = Dataset(X,y)

In [47]:
len(dataset)

768

In [52]:
# Load the data to your dataloader for batch processing and shuffling
# use torch.utils.data


data_loader=torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=32,
                                           shuffle=True)   # shuffle the data

In [54]:
len(data_loader)  #768/32 =24 batches

24

In [55]:
for (X,y) in data_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(X.shape))
    print("Labels:  {}".format(y.shape))
    break

For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


## Building the Network

![alt text](aaaa.png "Title")

In [60]:
import torch.nn as nn

In [62]:
class Model(nn.Module):  # inherit from nn module
    def __init__(self, input_features,output_features):   # initialization function .give the input and o/p features
        super(Model, self).__init__()                     # Super is used to pass to nn.Module to Model class 
        self.fc1 = nn.Linear(input_features, 5)         # nn.Linear to create a layer.defining first layer. takes i/p and o/p 5 neurons
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, output_features)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
        
    def forward(self, X):             ## tanh actv function for hidden layers and sigmoid funciton for output layer
        out = self.fc1(X)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out


$H_{p}(q)=-\frac{1}{N} \sum_{i=1}^{N} y_{i} \cdot \log \left(p\left(y_{i}\right)\right)+\left(1-y_{i}\right) \cdot \log \left(1-p\left(y_{i}\right)\right)$


cost = -(Y * torch.log(hypothesis) + (1 - Y) * torch.log(1 - hypothesis)).mean()

In [67]:
# Create the network (an object of the Net class)

net = Model(7,1)


In [68]:
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch

criterion = torch.nn.BCELoss(size_average=True) 

C:\Users\Vikee\anaconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [71]:
# We will use SGD with momentum with a learning rate of 0.1

optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

#net.parameters is to access the weights. parameter is the weights from net.those attributes are defined in init above
# lr - learning rate
# specify value for momentum as 0.9
## torch.optim.tab shows all optimizers available

## Training the Network

there is no direct training method in pytorch. we need to write it step by step

In [26]:
# in pytorch we train using for loop. we cannot use fit here. 
# set a range to epoch and for each epoch , load the data from data loader


num_epochs = 200
for epoch in range(num_epochs):
    for inputs,labels in data_loader:
        inputs = inputs.float()
        labels = labels.float()
        
        # Feed Forward. when we pass the inputs, python automatically does the forward function. we forwarded i/p to network
        output = net(inputs)
        
        # calculate loss. use the criterion loss defined above. pass actual (labels) and predicted (output from prev step)
        loss = criterion(output, labels)
        
        # w <-- w - lr * gradient -- we need to clear this gradient buffer. we don't want to accumulate gradients
        optimizer.zero_grad()
        
        # Backpropagation 
        loss.backward()
        
        # Weight Update: w <-- w - lr * gradient . done by .step()
        optimizer.step()
    
    #Accuracy calculation. this is done inside epoch loop but outside the data loader loop
    # Since we are using a sigmoid, we will need to perform some thresholding
    
    output = (output>0.5).float()
    
    # mean of correct predictions
    accuracy = (output == labels).float().mean()
    
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

NameError: name 'data_loader' is not defined

In [ ]:
# for testing accuracy, split the data to train and test data at beginning and repeat the same for test data